## 1. introduction & setup

we finetune llama 3.2 3B instruct on a malayalam question answering dataset[1]

[1] indic qa malayalam dataset, https://huggingface.co/datasets/sepiatone/ai4bharat-indicqa-ml-202410

#### 1a. we first install some required libraries

In [1]:
!pip install -qU datasets
!pip install -qU bitsandbytes
!pip install -qU peft
!pip install -qU trl

#### 1b. we log into huggingface

In [2]:
from google.colab import userdata
import os

os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

## 2. prepare the dataset

#### 2a. load the dataset

In [3]:
from datasets import load_dataset_builder, load_dataset

ds = load_dataset("sepiatone/ai4bharat-indicqa-ml-202410", split = "train")


# test
print(ds)

Dataset({
    features: ['version', 'data'],
    num_rows: 247
})


#### 2b. prepare the dataset

In [4]:
def process_data_sample(example):
    processed_example = ()
    context = example["data"]["paragraphs"][0]["context"]

    for j in range(len(example["data"]["paragraphs"][0]["qas"])):
      question = example["data"]["paragraphs"][0]["qas"][j]["question"]
      answer = example["data"]["paragraphs"][0]["qas"][j]["answers"][0]["text"]


      # prepare the processed example for a History Question Answering System
      processed_example = processed_example + (
          f"{context}\n\n"
          f"ഉപയോക്തൃ ചോദ്യം:\n{question}\n\n"
          f"ഉത്തരം:\n{answer}\n\n",
      )

    processed_example = "".join(processed_example)

    return processed_example

In [5]:
ds = ds.map(lambda example: {'text': process_data_sample(example)}, remove_columns=['version', 'data'])


# test
for i in range(2):
  print(ds[i]["text"])

 . ഹരപ്പയുമായി യാതൊരു ബന്ധവും കാണുന്ന തരത്തിലല്ല അവയുടെ രീതി. ഹരപ്പൻ കോട്ടക്ക്‌ തെക്ക്‌ഭാഗത്തായി കണ്ടെടുത്ത  ശ്മശാനസംസ്കൃതിയുടെ ഭാഗങ്ങളാകട്ടെ ഹരപ്പൻ സംസ്കാരവുമായി പൊരുത്തപ്പെട്ടു പോവാത്തതും പ്രകടമായ വ്യത്യാസമുള്ളവയുമാണ്‌. . ഇതിനെ ഹരപ്പാനന്തരഘട്ടമായി ചരിത്രകാരന്മാർ കണക്കാക്കുന്നു. ഇവിടെ നിന്നും കിട്ടിയ മൺപാത്രങ്ങളും അവയിലെ ചിത്രലേഖനങ്ങളും ഹരപ്പൻ പ്രദേശത്തിനു പുറത്തു നിന്നുള്ള ഒരു ജനവിഭാഗത്തിന്റേത്‌ എന്ന് സംശയിക്കത്തക്കവിധം വ്യത്യസ്തങ്ങളായിരുന്നു. ഹരപ്പൻ അധിവാസകേന്ദ്രങ്ങളിലേക്ക്‌ ഏതോ പരദേശിജീവിതരീതിയുടെ കടന്നു കയറ്റത്തേയാണ് ഇത്‌ സൂചിപ്പിക്കുന്നത്‌. അടുത്തകാലത്തായി നടത്തിയ ഗവേഷണങ്ങളിൽ ശവശരീരങ്ങൾ കണ്ടെത്തിയത് കൂട്ടക്കൊലകാരണമല്ല മറിച്ച് രോഗങ്ങൾ മൂലം മരണപ്പെട്ടവരുടേതാണെന്നാണ്‌ തെളിഞ്ഞത്. അസ്ഥികൂടങ്ങളുടെ പഠനത്തിൽ നിന്ന് മരണം ഉപരോധാമോ മറ്റോ കാരണമായി പിടിപെട്ട അനീമിയ പോലുള്ള അസുഖങ്ങൾ മൂലമാണ്‌ ഉണ്ടായതെന്ന് കണ്ടെത്തിട്ടുണ്ട്. മറ്റൊരു കൂട്ടം ഗവേഷകരുടേ അഭിപ്രായത്തിൽ  ഹരപ്പൻ നാഗരികതയുടെ അന്ത്യം കാലാവസ്ഥാ വ്യതിയാനം മൂലമാണ്. തുടർച്ചയായ പ്രളയമോ, അതെത്തുടർന്നുണ്ടായ വനനശീകരണമോ ആയിരിക്കാം ഹരപ്പയുടെ പാധാന്

## 3. model configuration and loading

we load the model and set the configuration parameters

In [6]:
import torch
import bitsandbytes
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

model_name = 'meta-llama/Llama-3.2-3B-Instruct'

# Set the quantization configuration
# note: load_in_4bit: True, the weights are in 4-bit
#       bnb_4bit_compute_dtype: torch.bfloat16, the computation still happens in 16 or 32-bit
#       bnb_4bit_quant_type: "nf4", use NF4 for higher precision
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

#### 3b. setup the tokenizer

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

## 4. peft and lora configuration

In [8]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias='none',
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
)

# model = get_peft_model(model, peft_config)

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments

output_dir = "./llama-3.2-3b-sft-indicqa-ml"

# Training arguments
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    gradient_checkpointing=True,
)

# SFTTrainer arguments
trainer = SFTTrainer(
    model=model,
    train_dataset=ds,
    peft_config=peft_config,
    dataset_text_field='text',
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=None
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda

#### 4b. let's finetune on the dataset!

In [10]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
25,0.640600
50,0.606900


TrainOutput(global_step=62, training_loss=0.6189579425319549, metrics={'train_runtime': 458.295, 'train_samples_per_second': 0.539, 'train_steps_per_second': 0.135, 'total_flos': 4314538772004864.0, 'train_loss': 0.6189579425319549, 'epoch': 1.0})

## 5. post training

#### 5a. push to hugging face

In [11]:
trainer.push_to_hub(commit_message = "end of finetuning", model_name = "llama-3.2-3b-sft-indicqa-ml-v0.1")

adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

events.out.tfevents.1730719674.23a4df52558f.11080.0:   0%|          | 0.00/6.87k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.50k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sepiatone/llama-3.2-3b-sft-indicqa-ml/commit/fd6af3a170c43deb9fc62cc0c8448b1bd94a0e6e', commit_message='end of finetuning', commit_description='', oid='fd6af3a170c43deb9fc62cc0c8448b1bd94a0e6e', pr_url=None, pr_revision=None, pr_num=None)